In [21]:
# armazenar os documentos totais em variavel

import os
#rootdir = '/media/r2-d2/E0C494CDC494A6F6/doutorado/20_newsgroup'
rootdir = '/media/developer/DATA/doutorado/rec informacao/20_newsgroup'
docs = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #if os.path.basename(subdir) in ['alt.atheism', 'rec.sport.baseball']:
        with open(os.path.join(subdir, file), encoding='cp1252') as f:                
            contents = f.read()
            docs.append({ 'tema': os.path.basename(subdir), 'nome_arquivo': file, 'conteudo': contents  })

In [22]:
# baixando as libs para pre processamento

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

nltk.download('stopwords')

stopwords = set(stopwords.words('english'))

stemmer = PorterStemmer()
#nltk.download("wordnet")
#from nltk.stem import WordNetLemmatizer
#lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/developer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
def remove_header(text):
    # Find the first occurrence of '\n\n'
    pos = text.find('\n\n')

    # Remove everything after the first '\n\n'
    if pos >= 0:
        text = text[pos+2:]
    return text
    

In [24]:
def remove_email(text):
    pattern = r'\S+@\S+'
    # Remove email addresses from the string
    text = re.sub(pattern, '', text)
    return text


In [43]:
# pre processamento

    # Remover caracteres especiais e números
    #text = re.sub('[^A-Za-z\s\']+', '', text)

def preprocess_text(text):


    #text = remove_header(text)
    #text = remove_email(text)


    # Converter tudo para letras minúsculas
    text = text.lower()
    
    # Remover as tags HTML
    text = re.sub('^[^A-Za-z0-9 ]+$', '', text)
    
    # manter so texto
    #text = re.sub('[^A-Za-z ]+', '', text)

    first_blank_line_index = text.find('\n\n')
    first_part = text[:first_blank_line_index]
    words = text[first_blank_line_index + 2:]

    words_titulo = nltk.word_tokenize(first_part)

    words_titulo = [word for word in words_titulo if not re.match('^[^A-Za-z0-9]+$', word)]

    # remover palavras que tenha tamanho 1
    words_titulo = [word for word in words_titulo if len(word) > 2]

    first_part = ' '.join(words_titulo)

    
    # Tokenizar o texto em palavras
    words = nltk.word_tokenize(text)

    # remover palavras que so possua caractere especial
    words = [word for word in words if not re.match('^[^A-Za-z0-9]+$', word)]

    # remover palavras que tenha tamanho 1
    words = [word for word in words if len(word) > 2]

    
    # Remover as stopwords
    words = [word for word in words if word not in stopwords]
    
    # Aplicar o stemming
    words = [stemmer.stem(word) for word in words]
    #words = [lemmatizer.lemmatize(word) for word in words]
    #words = stemm_preserva_cabecalho(words)

    # Juntar as palavras em uma única string
    text = ' '.join(words)

    text = first_part + " " + text
  
    return text

In [26]:
# rodar o pre processamento

def preprocess_collection(docs):

    preprocessed_documents = []

    for document in docs:
        tema = document['tema']
        nome_arquivo = document['nome_arquivo']
        conteudo = document['conteudo']
        
        preprocessed_conteudo = preprocess_text(conteudo)
        
        preprocessed_document = {
            'tema': tema,
            'nome_arquivo': nome_arquivo,
            'conteudo': preprocessed_conteudo
        }
        
        preprocessed_documents.append(preprocessed_document)
        
    return preprocessed_documents

In [44]:
preprocessed_documents = preprocess_collection(docs)
frases = [o['conteudo'] for o in preprocessed_documents]

In [45]:
frases[851]

"path cantaloupe.srv.cs.cmu.edu das-news.harvard.edu noc.near.net howland.reston.ans.net usc elroy.jpl.nasa.gov news.aero.org gck from gck aero.org gregory kozlowski newsgroups alt.atheism subject you will all hell date apr 1993 14:25:34 gmt organization the aerospace corporation segundo lines message-id 1rgrcuinnp0i news.aero.org references 1rbt9f mdv usenet.ins.cwru.edu 1993apr25.182557.16307 wam.umd.edu nntp-posting-host aerospace.aero.org summary are there path cantaloupe.srv.cs.cmu.edu das-news.harvard.edu noc.near.net howland.reston.ans.net usc elroy.jpl.nasa.gov news.aero.org gck gck aero.org gregori kozlowski newsgroup alt.ath subject hell date apr 1993 14:25:34 gmt organ aerospac corpor segundo line message-id 1rgrcuinnp0i news.aero.org refer 1rbt9f mdv usenet.ins.cwru.edu 1993apr25.182557.16307 wam.umd.edu nntp-posting-host aerospace.aero.org summari hell n't anyon notic consensu realiti special case"

In [46]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split(" ") for doc in frases]

bm25 = BM25Okapi(tokenized_corpus)

In [47]:
# armazenar os documentos totais em variavel

import os
#rootdir = '/media/r2-d2/E0C494CDC494A6F6/doutorado/mini_newsgroups'
rootdir = '/media/developer/DATA/doutorado/rec informacao/mini_newsgroups'

docs_consultas = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        with open(os.path.join(subdir, file), encoding='cp1252') as f:                
            contents = f.read()
            docs_consultas.append({ 'tema': os.path.basename(subdir), 'nome_arquivo': file, 'conteudo': contents  })

In [48]:
dirs_abs = []
for subdir, dirs, files in os.walk(rootdir):
    if os.path.basename(subdir) != '20_newsgroup':
        dirs_abs.append(os.path.basename(subdir))

temas = {}
for d in dirs_abs:
    temas[d] = {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}

print(temas)    

{'mini_newsgroups': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'alt.atheism': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.graphics': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.os.ms-windows.misc': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.sys.ibm.pc.hardware': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.sys.mac.hardware': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.windows.x': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'misc.forsale': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.autos': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.motorcycles': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.sport.baseball': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.sport.hockey': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'sci.crypt': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'sci.electronics': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 

In [49]:
def calcula_map(ranking, tema):
    acertos = 0
    maps = []
    for i, doc in enumerate(ranking):
        if tema in doc:
            acertos +=1
            maps.append(acertos/(i+1))
    if len(maps) > 0:
        return mean(maps)
    else:
        print("não achou nada")
        return 0.0

In [50]:
doc_consultas_preproc = preprocess_collection(docs_consultas)

In [51]:
doc_consultas_preproc[0]['conteudo']

"newsgroups alt.atheism path cantaloupe.srv.cs.cmu.edu das-news.harvard.edu noc.near.net howland.reston.ans.net zaphod.mps.ohio-state.edu darwin.sura.net haven.umd.edu uunet mnemosyne.cs.du.edu nyx jcopelan from jcopelan nyx.cs.du.edu the one and only subject where are they now message-id 1993apr16.133830.19778 mnemosyne.cs.du.edu sender usenet mnemosyne.cs.du.edu netnews admin account organization salvation army draft board references 1qi156innf9n senator-bedfellow.mit.edu 1ql0d3 5vo dr-pepper.east.sun.com date fri apr 13:38:30 gmt lines newsgroup alt.ath path cantaloupe.srv.cs.cmu.edu das-news.harvard.edu noc.near.net howland.reston.ans.net zaphod.mps.ohio-state.edu darwin.sura.net haven.umd.edu uunet mnemosyne.cs.du.edu nyx jcopelan jcopelan nyx.cs.du.edu one subject message-id 1993apr16.133830.19778 mnemosyne.cs.du.edu sender usenet mnemosyne.cs.du.edu netnew admin account organ salvat armi draft board refer 1qi156innf9n senator-bedfellow.mit.edu 1ql0d3 5vo dr-pepper.east.sun.com d

In [37]:
# apenas para teste
tokenized_query = doc_consultas_preproc[101]['conteudo'].split(" ")
bm25.get_top_n(tokenized_query, frases, n=100)

In [53]:
from statistics import mean

from IPython.display import clear_output

# consultando
p10 = []
p20 = []
p50 = []
p100 = []
map_f = []
tema_atual = ""

for c, doc_consulta in enumerate(doc_consultas_preproc):
    clear_output(wait=True)
    print(c)
    #ranking = similaridade(doc_consulta['conteudo'], preprocessed_documents, idfs)
    tokenized_query = doc_consulta['conteudo'].split(" ")
    ranking = bm25.get_top_n(tokenized_query, frases, n=len(doc_consultas_preproc))
    acertos = 0
    
    map_f.append(calcula_map(ranking, doc_consulta['tema']))
    
    for i, doc in enumerate(ranking):
        #print(f"Documento {doc[0]} - Target {doc[1]} -->  {doc[2]} - Similaridade: {doc[3]}")
        #if doc[1] == doc_consulta['tema']:
        
        if doc_consulta['tema'] in doc:
            #print(f"{doc_consulta['tema']} e {doc}")
            acertos+=1
        if i+1 == 10:
            p10.append(acertos/10.0)            
        if i+1 == 20:
            p20.append(acertos/20.0)
        if i+1 == 50:
            p50.append(acertos/50.0)
        if i+1 == 100:
            p100.append(acertos/100.0) 
        #map.append(calcula_map(ranking, doc_consulta['tema']))
    if c > 0:
        if tema_atual != doc_consulta['tema'] or c == len(doc_consultas_preproc) - 1:
            temas[tema_atual]['p10'] = mean(p10)
            temas[tema_atual]['p20'] = mean(p20)
            temas[tema_atual]['p50'] = mean(p50)
            temas[tema_atual]['p100'] = mean(p100)
            temas[tema_atual]['map'] = mean(map_f)
            p10 = []
            p20 = []
            p50 = []
            p100 = []
            map = []

    tema_atual = doc_consulta['tema']
    # if c > 202:
    #     break    


1999


In [54]:
temas

{'mini_newsgroups': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0},
 'alt.atheism': {'p10': 0.907920792079208,
  'p20': 0.8514851485148515,
  'p50': 0.7803960396039604,
  'p100': 0.7288118811881188,
  'map': 0.5190474618373421},
 'comp.graphics': {'p10': 0.515,
  'p20': 0.433,
  'p50': 0.3998,
  'p100': 0.3756,
  'map': 0.40771429986613905},
 'comp.os.ms-windows.misc': {'p10': 0.644,
  'p20': 0.5585,
  'p50': 0.5062,
  'p100': 0.4765,
  'map': 0.40346701283654707},
 'comp.sys.ibm.pc.hardware': {'p10': 0.582,
  'p20': 0.4815,
  'p50': 0.4122,
  'p100': 0.3827,
  'map': 0.38080905134878906},
 'comp.sys.mac.hardware': {'p10': 0.657,
  'p20': 0.5535,
  'p50': 0.4468,
  'p100': 0.4104,
  'map': 0.37188495863300536},
 'comp.windows.x': {'p10': 0.63,
  'p20': 0.567,
  'p50': 0.5318,
  'p100': 0.4945,
  'map': 0.36934506444719634},
 'misc.forsale': {'p10': 0.498,
  'p20': 0.419,
  'p50': 0.4052,
  'p100': 0.3956,
  'map': 0.3707311977966054},
 'rec.autos': {'p10': 0.684,
  'p20': 0.565,
 

In [55]:
import json
with open('data_stemm_bm25_2.json', 'w') as f:
    json.dump(temas, f)

In [56]:
import pandas as pd

df = pd.DataFrame(temas)
df =df.transpose()
df.drop('mini_newsgroups')


,p10,p20,p50,p100,map
alt.atheism,0.907921,0.851485,0.780396,0.728812,0.519047
comp.graphics,0.515000,0.433000,0.399800,0.375600,0.407714
comp.os.ms-windows.misc,0.644000,0.558500,0.506200,0.476500,0.403467
comp.sys.ibm.pc.hardware,0.582000,0.481500,0.412200,0.382700,0.380809
comp.sys.mac.hardware,0.657000,0.553500,0.446800,0.410400,0.371885
comp.windows.x,0.630000,0.567000,0.531800,0.494500,0.369345
misc.forsale,0.498000,0.419000,0.405200,0.395600,0.370731
rec.autos,0.684000,0.565000,0.476600,0.421000,0.366412
rec.motorcycles,0.794000,0.687000,0.560400,0.475600,0.367720
rec.sport.baseball,0.772000,0.690000,0.570800,0.502400,0.370242


In [55]:
df

,p10,p20,p50,p100,map
mini_newsgroups,0.000000,0.000000,0.000000,0.000000,0.000000
alt.atheism,0.916832,0.867327,0.795644,0.731089,0.850311
comp.graphics,0.591000,0.525000,0.480200,0.439500,0.708645
comp.os.ms-windows.misc,0.694000,0.613000,0.548800,0.504800,0.683341
comp.sys.ibm.pc.hardware,0.650000,0.588500,0.505800,0.454900,0.662263
comp.sys.mac.hardware,0.690000,0.600500,0.507200,0.463200,0.650734
comp.windows.x,0.693000,0.614000,0.554200,0.500300,0.648074
misc.forsale,0.580000,0.516000,0.478800,0.448200,0.637965
rec.autos,0.763000,0.682000,0.590600,0.511500,0.644341
rec.motorcycles,0.836000,0.755000,0.653200,0.561800,0.654888
